In [5]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 47.3948,
	"longitude": 0.704,
	"hourly": ["temperature_2m", "rain", "weather_code", "wind_speed_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["weather_code"] = hourly_weather_code
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 47.400001525878906°N 0.6999998092651367°E
Elevation 51.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m  rain  weather_code  \
0   2024-03-07 00:00:00+00:00        3.982500   0.0           0.0   
1   2024-03-07 01:00:00+00:00        3.632500   0.0           0.0   
2   2024-03-07 02:00:00+00:00        3.282500   0.0           0.0   
3   2024-03-07 03:00:00+00:00        2.982500   0.0           0.0   
4   2024-03-07 04:00:00+00:00        2.732500   0.0           0.0   
..                        ...             ...   ...           ...   
163 2024-03-13 19:00:00+00:00        9.868999   0.0           3.0   
164 2024-03-13 20:00:00+00:00        9.568999   0.0           3.0   
165 2024-03-13 21:00:00+00:00        9.318999   0.0           3.0   
166 2024-03-13 22:00:00+00:00        9.068999   0.0          45.0   
167 2024-03-13 23:00:00+00:00        8.868999   0.0          45.0   

     wind_speed_10m  
0          6.287130  


In [6]:
hourly_dataframe

,date,temperature_2m,rain,weather_code,wind_speed_10m
0,2024-03-07 00:00:00+00:00,3.982500,0.0,0.0,6.287130
1,2024-03-07 01:00:00+00:00,3.632500,0.0,0.0,6.379216
2,2024-03-07 02:00:00+00:00,3.282500,0.0,0.0,6.638072
3,2024-03-07 03:00:00+00:00,2.982500,0.0,0.0,7.421590
4,2024-03-07 04:00:00+00:00,2.732500,0.0,0.0,8.209263
...,...,...,...,...,...
163,2024-03-13 19:00:00+00:00,9.868999,0.0,3.0,0.804985
164,2024-03-13 20:00:00+00:00,9.568999,0.0,3.0,0.804985
165,2024-03-13 21:00:00+00:00,9.318999,0.0,3.0,0.804985
166,2024-03-13 22:00:00+00:00,9.068999,0.0,45.0,0.720000
